In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartForConditionalGeneration, BartConfig
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/NanumFont/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = T5TokenizerFast.from_pretrained(model_dir) 

# https://huggingface.co/paust/pko-t5-base
# transformers 의 API 를 사용하여 접근 가능합니다. 
# tokenizer 를 사용할때는 T5Tokenizer 가 아니라 T5TokenizerFast 를 사용해주십시오. 
# model 은 T5ForConditionalGeneration 를 그대로 활용하시면 됩니다.

In [ ]:
config_data = {
    "general": {
        "data_path": "your_path", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": f"{model_dir}", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 150,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "unk_token": f"{tokenizer.unk_token}",
        "pad_token": f"{tokenizer.pad_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#',
                           '#DateOfBirth#', '#CarNumber#', '#Email#', '#CardNumber#', '#Address#', '#SSN#', '#PhoneNumber#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 50,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 4, # 50
        "per_device_eval_batch_size": 16, # 32
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 16, # 몇 개의 작은 배치를 합쳐서 큰 배치처럼 처리할지를 결정하는 파라미터: 메모리 제한 극복, 더 나은 성능
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 7,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 150,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 5, # 3
        "early_stopping_threshold": 0.0001, # 0.001
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "entity_name",
        "project": "project_name",
        "name": "name"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 150,
        "num_beams": 4,
        "batch_size" : 16, # 32
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.unk_token}", f"{tokenizer.pad_token}"]
    }
}

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 이곳에 사용자가 저장한 데이터 dir 설정하기
loaded_config['general']['data_path'] = "../../data"